In [1]:
# Ralph Mouawad - ID 202204667
# INDE 504 - Simulation - Dr. Bacel Maddah
# Lab 10 - Simulation with SimPy

In [3]:
import simpy
import random

In [21]:
# Exercise 1

class VisaApplicationCenter:
    def __init__(self, env, arrival_rate, num_employees, doc_sub_time, interview_time, payment_time, biometric_time):
        self.env = env
        self.arrival_rate = arrival_rate
        self.num_employees = num_employees
        self.doc_sub_time = doc_sub_time
        self.interview_time = interview_time
        self.payment_time = payment_time
        self.biometric_time = biometric_time
        self.applications_arrived = 0
        self.applications_processed = 0
        self.employees = [simpy.Resource(env) for _ in range(num_employees)]
        self.waiting_time = 0

    def process_applications(self, env): # model the visa application process
        arrival_time = env.now
        with self.employees[0].request() as req:
            yield req
            yield env.timeout(random.expovariate(1/self.doc_sub_time))
        with self.employees[1].request() as req:
            yield req
            yield env.timeout(random.expovariate(1/self.interview_time)) 
        with self.employees[2].request() as req:
            yield req
            yield env.timeout(random.expovariate(1/self.payment_time))
        with self.employees[3].request() as req:
            yield req
            yield env.timeout(random.expovariate(1/self.biometric_time))
        self.applications_processed += 1 # one app has been processed
        self.waiting_time += env.now - arrival_time
    
    def application_arrival(self, env):
        while True:
            interarrival_time = random.expovariate(self.arrival_rate)
            yield env.timeout(interarrival_time)
            self.applications_arrived +=1
            env.process(self.process_applications(env))
            

In [44]:
env = simpy.Environment()
visa_office = VisaApplicationCenter(env, arrival_rate=10, num_employees=4, doc_sub_time=1/4, interview_time=1/2, payment_time=1/10, biometric_time=1/5)
env.process(visa_office.application_arrival(env))
env.run(until = 8)

In [45]:
average_waiting_time = visa_office.waiting_time/visa_office.applications_processed
print(f"Total applications arrived: {visa_office.applications_arrived}")
print(f"Total applications processed: {visa_office.applications_processed}")
print(f"Average waiting time for applications: {average_waiting_time: .2f} hours")


Total applications arrived: 95
Total applications processed: 12
Average waiting time for applications:  4.06 hours


In [4]:
# Exercise 2

class CoffeeShopCenter:
    def __init__(self, env, arrival_rate, num_employees, processing_rate):
        self.env = env
        self.arrival_rate = arrival_rate
        self.num_employees = num_employees
        self.processing_rate = processing_rate
        self.applications_arrived = 0
        self.applications_processed = 0
        self.waiting_time = 0
        self.application_process = simpy.Resource(env, capacity=num_employees)
    
    def process_application(self, env):
        arrival_time = self.env.now
        with self.application_process.request() as request:
            yield request
            self.applications_processed +=1 
            yield self.env.process(self.process_steps(env))
            end_time = self.env.now
            self.waiting_time += end_time - arrival_time

    def process_steps(self, env):
        total_processing_time = random.gammavariate(3, 1/self.processing_rate)
        yield self.env.timeout(total_processing_time)

    def application_arrival(self, env):
        while True:
            interarrival_time = random.expovariate(self.arrival_rate)
            yield self.env.timeout(interarrival_time)
            self.applications_arrived += 1
            self.env.process(self.process_application(env))


In [5]:
env = simpy.Environment()
coffee_shop = CoffeeShopCenter(env, arrival_rate=35, num_employees=3, processing_rate=5)
env.process(coffee_shop.application_arrival(env))
env.run(until=8)

In [6]:
total_arrival = coffee_shop.applications_arrived
total_processed = coffee_shop.applications_processed
average_waiting_time = coffee_shop.waiting_time/coffee_shop.applications_processed
print(f"Total customers arrived: {coffee_shop.applications_arrived}")
print(f"Total customers processed: {coffee_shop.applications_processed}")
print(f"Average waiting time for customers: {average_waiting_time: .2f} hours")


Total customers arrived: 283
Total customers processed: 39
Average waiting time for customers:  3.17 hours
